# Data Preprocessing

- Build a data processing pipeline that takes the raw data and transforms them into a NN trainable dataset


## Imports

In [3]:
from IPython.display import Javascript
import ipywidgets as widgets

# Data
import numpy as np
import pandas as pd

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Preprocessing - Part 1

In [4]:
class PreprocessingPipeline():
    '''
    -> Outlier handling
    -> Missing-value handling
    -> Power Transform (Cox Box, Yao Johnsen)
    -> Categorical data (One hot encoding, Label Encoding)
    '''
    
    def __init__ (self, df, fill_type) -> None:
        self.fill_type = fill_type
        self.feature_names = 
        
    def fit():
        pass
    
    def transform():
        pass
    
    def batch_transform():
        pass
    
    # Outlier handling
    def __handle_outliers():
        pass
    
    def __detect_outliers_iqr():
        pass
    
    # Missing value handling
    def __fill_missing_values():
        pass
    
    # Power transformations
    def __power_transform():
        pass
    
    # Encoding of the non-numeric variables
    def __encode_variables():
        pass
    

# Preprocessing - Part 2

In [ ]:
class FeatureScaling():